# A/B Dashboard Health Notebook

This lightweight notebook pings Supabase RPCs used by the dashboard, summarizes results, and logs a run to the database via the `log_analytics_run` RPC.

Inputs (env):
- `PUBLIC_SUPABASE_URL`
- `PUBLIC_SUPABASE_ANON_KEY`

In [5]:
import os, time, json, textwrap
import requests
from pathlib import Path

# Load env from .env if not in environment (VS Code local runs)
def _load_env_from_files(candidates):
    for p in candidates:
        try:
            path = Path(p)
            if path.exists():
                for line in path.read_text().splitlines():
                    line = line.strip()
                    if not line or line.startswith('#') or '=' not in line:
                        continue
                    k, v = line.split('=', 1)
                    os.environ.setdefault(k.strip(), v.strip())
                break
        except Exception:
            pass

_load_env_from_files(['.env', '../.env', '../../.env'])

SUPABASE_URL = os.environ.get('PUBLIC_SUPABASE_URL')
SUPABASE_ANON_KEY = os.environ.get('PUBLIC_SUPABASE_ANON_KEY')
assert SUPABASE_URL and SUPABASE_ANON_KEY, 'Set PUBLIC_SUPABASE_URL and PUBLIC_SUPABASE_ANON_KEY in the environment or .env'

headers = {
    'apikey': SUPABASE_ANON_KEY,
    'Authorization': f'Bearer {SUPABASE_ANON_KEY}',
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}
def postrpc(path, body=None, expect_json=True):
    url = f"{SUPABASE_URL}/rest/v1/rpc/{path}"
    r = requests.post(url, headers=headers, data=json.dumps(body if body is not None else {}))
    r.raise_for_status()
    if not expect_json or r.status_code == 204 or not r.text.strip():
        return None
    return r.json()

def getview(path_with_query):
    url = f"{SUPABASE_URL}/rest/v1/{path_with_query}"
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json()

In [6]:
job_name = 'ab_dashboard_health'
t0 = time.time()
status = 'ok'
message_lines = []
try:
    overview = postrpc('variant_overview', None)
    funnel = getview('v_conversion_funnel?select=*')
    dist = postrpc('completion_time_distribution', {})
    comps = postrpc('recent_completions', {'limit_count': 5})

    message_lines.append(f"overview rows: {len(overview)}")
    message_lines.append(f"funnel rows: {len(funnel)}")
    message_lines.append(f"dist keys: {list((dist[0] if isinstance(dist, list) and dist else dist).keys())}")
    message_lines.append(f"recent_completions: {len(comps)}")
except Exception as e:
    status = 'fail'
    message_lines.append(f'error: {e!r}')

duration_ms = int((time.time() - t0) * 1000)
summary = ' | '.join(message_lines)
print(f'status={status} duration_ms={duration_ms} summary={summary}')

status=ok duration_ms=707 summary=overview rows: 1 | funnel rows: 8 | dist keys: ['variant_a_times', 'variant_b_times'] | recent_completions: 5


In [7]:
# Log the run to the DB via SECURITY DEFINER RPC (no service key needed)
try:
    postrpc('log_analytics_run', {
        'job_name': job_name,
        'status': status,
        'duration_ms': duration_ms,
        'message': summary[:2000]
    }, expect_json=False)
    print('logged to analytics_run_log')
except Exception as e:
    print('log failed:', e)

logged to analytics_run_log
